In [144]:
import pandas as pd 
import requests
import json
from datetime import datetime

In [145]:
JSON_URL = "https://wowtokenprices.com/history_prices_full.json"

In [146]:
def request_token_price(request):
    response = requests.get(request)
    response_json = response.json()
    return response_json['eu']

In [147]:
df = pd.DataFrame(request_token_price(JSON_URL))

In [148]:
df['time_of_last_change_utc_timezone']=df['time'].apply(lambda x: datetime.fromtimestamp(x))
df['day']=df['time_of_last_change_utc_timezone'].apply(lambda x: x.weekday())
df['hour']=df['time_of_last_change_utc_timezone'].apply(lambda x: x.hour)
df['time_of_last_change_utc_timezone']=pd.to_datetime(df['time_of_last_change_utc_timezone'])

In [149]:
df=df.set_index('time_of_last_change_utc_timezone',drop=False)
df = df.sort_index()

In [150]:
std = lambda x: (x.std())
df['7_day_std']=df['price'].rolling('7d').apply(std)
df['30_day_std']=df['price'].rolling('30d').apply(std)
df['7_day_mean']=df['price'].rolling('7d').mean()
df['30_day_mean']=df['price'].rolling('30d').mean()
df['7_day_low']=df['price'].rolling('7d').min()
df['30_day_low']=df['price'].rolling('30d').min()
df['7_day_high']=df['price'].rolling('7d').max()
df['30_day_high']=df['price'].rolling('30d').max()
df['1_day_high']=df['price'].rolling('1d').max()
df['1_day_low']=df['price'].rolling('1d').min()
df['last_change']=df['price'].diff(periods=2)
df=df.dropna()

In [151]:
df=df.rename(columns = {'price':'current_price'}, inplace = False)
df

,current_price,time,time_of_last_change_utc_timezone,day,hour,7_day_std,30_day_std,7_day_mean,30_day_mean,7_day_low,30_day_low,7_day_high,30_day_high,1_day_high,1_day_low,last_change
time_of_last_change_utc_timezone,,,,,,,,,,,,,,,,
2015-04-21 22:35:33,35704,1429648533,2015-04-21 22:35:33,1,22,352.001894,352.001894,35351.333333,35351.333333,35000.0,35000.0,35704.0,35704.0,35704.0,35000.0,704.0
2015-04-21 22:49:56,36061,1429649396,2015-04-21 22:49:56,1,22,456.629226,456.629226,35528.750000,35528.750000,35000.0,35000.0,36061.0,36061.0,36061.0,35000.0,711.0
2015-04-21 23:05:02,36422,1429650302,2015-04-21 23:05:02,1,23,562.104795,562.104795,35707.400000,35707.400000,35000.0,35000.0,36422.0,36422.0,36422.0,35000.0,718.0
2015-04-21 23:20:38,36786,1429651238,2015-04-21 23:20:38,1,23,668.330582,668.330582,35887.166667,35887.166667,35000.0,35000.0,36786.0,36786.0,36786.0,35000.0,725.0
2015-04-21 23:35:11,37154,1429652111,2015-04-21 23:35:11,1,23,775.556666,775.556666,36068.142857,36068.142857,35000.0,35000.0,37154.0,37154.0,37154.0,35000.0,732.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-07 11:04:45,162625,1610013885,2021-01-07 11:04:45,3,11,3106.778300,3038.115800,160678.334375,161467.829422,155921.0,155921.0,169864.0,172926.0,162625.0,155921.0,372.0
2021-01-07 11:24:45,162730,1610015085,2021-01-07 11:24:45,3,11,3104.033162,3018.758424,160684.725857,161458.627256,155921.0,155921.0,169864.0,172826.0,162730.0,155921.0,284.0
2021-01-07 11:44:45,162514,1610016285,2021-01-07 11:44:45,3,11,3100.870569,2999.515718,160690.406832,161449.320397,155921.0,155921.0,169864.0,172613.0,162730.0,155921.0,-111.0


In [152]:
df.to_csv("Token-prices.csv",index=False)

In [157]:
df['last_change'].max()

46481.0